 <h2> Domain.com.au Web Scraping</h2>
                             <img src="http://www.domain.com.au/Content/Files/misc/domainLogoHead.gif">

IMPORTING LIBRARIES

In [1]:
# HTML extraction & Parsin
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib
import math
import os


# Calculate Latitude & Longitude
from geopy.geocoders import Nominatim
# Calculate Distance from Latitude & Longitude
from math import sin, cos, sqrt, atan2, radians
import emailsend



SET GLOBAL PARAMETERS

In [2]:

region_list = [
'melbourne-region-vic',
'sydney-region-nsw',
'brisbane-region-qld',
'adelaide-region-sa',
'perth-metropolitan-eastern-suburbs-wa',
'perth-metropolitan-western-suburbs-wa',
'perth-metropolitan-northern-suburbs-wa',
'perth-metropolitan-southern-suburbs-wa',
]




price_range_list = [
 '000000-150000',
# '150000-200000',
# '200000-250000',
# '250000-300000',
# '300000-350000',
# '350000-400000',
# '400000-450000',
# '450000-500000',
# '500000-550000',
# '550000-600000',
# '600000-650000',
# '650000-700000',
# '700000-750000',
# '750000-800000',
# '800000-850000',
# '850000-900000',
# '900000-950000',
# '950000-1000000',
# '1000000-1050000',
# '1050000-1100000',
# '1100000-1150000',
# '1150000-1200000',
# '1200000-1250000',
# '1250000-1300000',
# '1300000-1350000',
# '1350000-1400000',
# '1400000-1450000',
# '1450000-1500000',
# '1500000-1550000',
# '1550000-1600000',
# '1600000-1650000',
# '1650000-1700000',
# '1700000-1750000',
# '1750000-1800000',
# '1800000-1850000',
# '1850000-1900000',
# '1900000-1950000',
# '1950000-2000000',
# '2000000-2050000',
# '2050000-2100000',
# '2100000-2150000',
# '2150000-2200000',
# '2200000-2250000',
# '2250000-2300000',
# '2300000-2350000',
# '2350000-2400000',
# '2400000-2450000',
# '2450000-2500000',
# '2500000-2550000',
# '2550000-2600000',
# '2600000-2650000',
# '2650000-2700000',
# '2700000-2750000',
# '2750000-2800000',
# '2800000-2850000',
# '2850000-2900000',
# '2900000-2950000',
# '2950000-3000000',
# '3000000-3200000',
# '3200000-3400000',
# '3400000-3600000',
# '3600000-3800000',
# '3800000-4000000',
# '4000000-4200000',
# '4200000-4400000',
# '4400000-4600000',
# '4600000-4800000',
# '4800000-5000000',
# '5000000-5200000',
# '5200000-5400000',
# '5400000-5600000',
# '5600000-5800000',
# '5800000-6000000',
# '6000000-6200000',
]

urls = []
# Change the index to change the city
region = region_list[0]


city = {region}.split('-')[0].capitalise()
print(f"CHOSEN CITY: {city}")
for p_range in price_range_list:
    urls.append(f"https://www.domain.com.au/sold-listings/{region}/house/?price={p_range}&excludepricewithheld=1&landsize=1-any&ssubs=0&page=")


house_list = []



CHOSEN CITY: melbourne-region-vic


API CALL VIA PROXY

In [ ]:
for url in urls:
    max_pages = 50
    for x in range(1,max_pages+1):
        full_url = 'https://app.scrapingbee.com/api/v1/?api_key=T6K98ENA0TKGTK4BK6LU7JCATYMR52A6YXD0QGSYH5B0NGIOE4X5PUQ4LDEJK77YETEX3XORL6S1ITDB&url='+urllib.parse.quote_plus(url+str(x))
        print(full_url)
        html_text = requests.get(full_url).text
        soup = BeautifulSoup(html_text,'lxml')
        if x ==1:
            max_pages = math.ceil(int(soup.find(attrs = {'data-testid':'summary'}).text.split(' ')[0])/20)
            print(str(max_pages)+ ' is the number of max pages')
            
        houses = soup.find_all('li',class_= 'css-1qp9106')

        for house in houses:

            # Extract Data Sold & Method of Sale
            date_method = house.find('span', class_='css-1nj9ymt').text
            date_method_split = date_method.split(" ")
            date = f"{date_method_split[-3]} {date_method_split[-2]} {date_method_split[-1]}"
            if "auction" in date_method:
                method = 'auction'
            elif "private treaty" in date_method:
                method = 'private treaty'
            else:
                method = 'Another Method'

            # Extract Address details
            try:
    
                street = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line1'}).text.replace(',','')
                
            except:
                print('ISSUE FOUND WITH STREET')
                street = 'No Street Found'
            
            try:
                 address = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line2'}).text
                 address_split = address.split(' ')
                 postcode = address_split[-1]
                 state = address_split[-2]
                 if len(address_split) == 4:
                    suburb = f"{address_split[0]} {address_split[1]}"
                 elif len(address_split) == 5:
                    suburb = f"{address_split[0]} {address_split[1]} {address_split[2]}"
                 else:
                    suburb = address_split[0]
                
            except:
                print('ISSUE FOUND WITH ADDRESS')
            
               
            

            
            price = house.find('p', class_='css-mgq8yx').text.replace(' price from APM PriceFinder','').replace('$','').replace(',','')
            type = house.find('span', class_='css-693528').text.replace('/ Unit / Flat','')

            try:
                features = house.find('div', class_ ='css-18biwo', attrs = {'data-testid':'property-features-wrapper'})
                bedrooms = features.span.span.text[0]
                if bedrooms == '−' or bedrooms is None:
                    bedrooms = 0
                bathrooms = features.span.next_sibling.span.text[0]
                if bathrooms == '−' or bathrooms is None:
                    bathrooms = 1
                cars = features.span.next_sibling.next_sibling.span.text[0]
                if cars == '−':
                    cars = 0
                if len(list(features.children)) == 4:
                    area = features.span.next_sibling.next_sibling.next_sibling.span.text
                else:
                    area = 0
            except:
                bedrooms = 0
                bathrooms = 0
                cars = 0
                area = 0
                print(f"No features found for {street} {address}")

            agent = house.find('div', class_='css-6yavch',attrs={'data-testid':'listing-card-lazy-image'})
            if agent is None:
                agent = 'UNKNOWN'
            elif agent.img['alt'][0] == 'P':
                agent = 'UNKNOWN'
            else:
                agent = agent.img['alt'].replace('Logo for ','')
            
            
            
        
            house_details = {
                'Street' : street,
                'Address' : address,
                'Suburb' : suburb,
                'State' : state,
                'Postcode' : postcode,
                'Type' : type,
                'Date': date,
                'Method' : method,
                'Price' : price,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Cars' : cars,
                'Area' : area,
                'Agent' : agent
            }
            if (bedrooms == 0 and bathrooms == 0 and cars == 0 and area == 0) or street == 'No Street Found':
                print(f"{street} {address} house was NOT added")
            else:
                house_list.append(house_details)
               
        print(str(len(house_list))+' houses added so far')


SAVING PANDAS DF TO CSV. PLEASE NOTE: WE ARE APPENDING TO CSV

In [ ]:
df = pd.DataFrame(house_list)
if not os.path.exists(f"../data/{city}/"):
    os.mkdir(f"../data/{city}/")
df.to_csv(f"../data/{city}/{city}_raw.csv",header = True, index = False)
df.tail(6)

Creating Latitude & Longitude columns from address

In [ ]:

df = pd.read_csv(f"../data/{city}/{city}_raw.csv")

def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long


vect_get_coordinates = np.vectorize(get_coordinates)



for i in range(0,len(df.index),1000):
    df_seg = df[i:i+1000]
    df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
    df_seg.to_csv(f"../data/{city}/{city}_latlon.csv",mode='a',header = False,index=False)
    print(f"PROCESSED: {len(df_seg)} RECORDS OUT OF TOTAL {len(df.index)}")
    print(f"PROGRESS: {len(df_seg)/len(df.index)} % COMPLETED")

    

CALCULATE DISTANCE FROM CITY by vectorization function and using Latitude and longitude

In [ ]:


df = pd.read_csv(f"../data/{city}/{city}_latlon.csv")

suburb_of_city = f"{city.split('-')[0]}"
la2,lo2 = get_coordinates('No Street Found', suburb_of_city)


def calc_distance(la1,lo1):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(la1)
    lon1 = radians(lo1)
    lat2 = radians(la2)
    lon2 = radians(lo2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
   

vec_cal_distance = np.vectorize(calc_distance)
df['Distance']  = vec_cal_distance(df.Latitude, df.Longitude)
df.to_csv(f"../data/{city}/{city}_distance.csv",header = True,index=False)
df.head(20)

In [13]:
df = pd.read_csv(f"../data/{city}/{city}_distance.csv")

def convert_area(area):
    # approximate radius of earth in km
    if ',' in area:
        area = area.replace(',','')
    if 'm²' in area:
        area = area.replace('m²','')
    if 'ha' in area:
        area = 10000*float(area.replace('ha',''))
    return area
   

vec_convert_area = np.vectorize(convert_area)
df['Area']  = vec_convert_area(df.Area)


df.to_csv(f"../data/{city}/{city}_area.csv",header = True,index=False)

Send Email of the collected & transformed data

In [ ]:

emailsend.run(f"../data/{city}/{city}_distance.csv")